In [34]:
import shutil
import os
from zipfile import ZipFile
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
# from diffusers import StableDiffusionPipeline
# import torch
import keras


In [27]:
# Get file from google drive
destination = os.getcwd()
zip_file_path = './Voitures.zip'
source = 'drive/MyDrive/Shared/Voitures.zip'
if not os.path.exists(zip_file_path):
  shutil.copy2(source, destination)
print(os.listdir())

['.config', 'Voitures.zip', '.ipynb_checkpoints', 'drive', 'sample_data']


In [28]:
# Unzip file

with ZipFile(zip_file_path, 'r') as zObject:

    # Extracting all the members of the zip
    # into a specific location.
    zObject.extractall(path=".")

In [29]:
# Visualize CSV content
file_path = "./vehicles.csv"
df = pd.read_csv(file_path,  usecols=['price', 'year', 'manufacturer', 'model', 'condition','cylinders', 'size',
                                     'type', 'paint_color', 'state', 'description'], engine='python')

In [30]:
# Supprimer les valeurs manquantes
df.dropna(inplace=True)

# Afficher un aperçu
print(df.head())

    price    year manufacturer                 model  condition    cylinders  \
31  15000  2013.0         ford             f-150 xlt  excellent  6 cylinders   
55  19900  2004.0         ford       f250 super duty       good  8 cylinders   
59  14000  2012.0        honda               odyssey  excellent  6 cylinders   
65  22500  2001.0         ford                  f450       good  8 cylinders   
73  15000  2017.0        dodge  charger rt 4dr sedan  excellent  8 cylinders   

         size      type paint_color  \
31  full-size     truck       black   
55  full-size    pickup        blue   
59  full-size  mini-van      silver   
65  full-size     truck       white   
73   mid-size     sedan        grey   

                                          description state  
31  2013 F-150 XLT V6 4 Door. Good condition. Leve...    al  
55  Here I have a unmolested Ford F-250 6.0 power ...    al  
59  Readily available. Excellent condition, clean ...    al  
65  Ford Super Duty F450 , XL with K

In [31]:
categorical_cols = ['manufacturer', 'model', 'condition', 'cylinders', 'size', 'type', 'paint_color', 'state']

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [32]:
X = df.drop(columns=['price', 'description'])  # Features
y = df['price']  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:

model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # Sortie unique : le prix
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
1027/1027 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 428890333904896.0000 - mae: 143892.1250 - val_loss: 171282816.0000 - val_mae: 9872.0654
Epoch 2/20
1027/1027 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 128969152135168.0000 - mae: 76766.3125 - val_loss: 714832640.0000 - val_mae: 23447.2207
Epoch 3/20
1027/1027 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 220150695460864.0000 - mae: 116103.6797 - val_loss: 1408378624.0000 - val_mae: 33604.4648
Epoch 4/20
1027/1027 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 137353632940032.0000 - mae: 76124.0781 - val_loss: 2813261056.0000 - val_mae: 48230.3477
Epoch 5/20
1027/1027 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 104739312238592.0000 - mae: 112759.5625 - val_loss: 5289832960.0000 - val_mae: 66549.5703
Epoch 6/20
1027/1027 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 185694202888192.0000 - mae: 115319.0625 - val_loss: 1986925440.0000 - val_mae: 40191.5430
Epoch 7/20
1027/1027 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 263036178792448.0000 - mae: 139

AttributeError: module 'tensorflow.keras' has no attribute 'saving'

In [36]:
# Sauvegarde du modèle
model_path="v1.keras"
keras.saving.save_model(model, model_path)

In [ ]:
# Charger le modèle de diffusion (code demo)
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")
pipe.to("cuda")  # Utiliser uniquement le GPU

def generate_car_image(description, predicted_price):
    prompt = f"Voiture {description}, estimée à {int(predicted_price)}€, design moderne et détaillé"
    image = pipe(prompt).images[0]
    image.show()
    image.save("generated_car.png")

# Exemple : Prédire le prix et générer une image
predicted_price = model.predict(X_test.iloc[0:1])[0][0]
description = df.iloc[0]['description']
generate_car_image("Mercedes benz du ciel", 150)

Couldn't connect to the Hub: 504 Server Error: Gateway Time-out for url: https://huggingface.co/api/models/CompVis/stable-diffusion-v1-4.
Will try to load from local cache.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]